# Q1

In [1]:
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv
#I used the csv module as i have to read data from csv module to perform map reduce 
import pandas as pd

In [2]:
file = pd.read_csv('Movies.csv')

# counting number of lines
print("Number of lines present before deleting first two rows : ", len(file))

Number of lines present before deleting first two rows :  1660


In [3]:
#This code will delete the first row of csv file as it gives information about category of column which is not required in map reduce
import csv

filename = "Movies.csv"
row_to_delete = 0  # Index of the row to delete

rows_to_keep = []
with open(filename, "r") as csv_file:
    reader = csv.reader(csv_file)
    for i, row in enumerate(reader):
        if i != row_to_delete:
            rows_to_keep.append(row)

with open(filename, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(rows_to_keep)

In [4]:
#This code will delete the new first row of csv file as it gives information about data type of column which is not required in map reduce
import csv

filename = "Movies.csv"
row_to_delete = 0  # Index of the row to delete

rows_to_keep = []
with open(filename, "r") as csv_file:
    reader = csv.reader(csv_file)
    for i, row in enumerate(reader):
        if i != row_to_delete:
            rows_to_keep.append(row)

with open(filename, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(rows_to_keep)

In [5]:
file = pd.read_csv('Movies.csv')

# counting number of lines
print("Number of lines present after deleting first two rows : ", len(file))

Number of lines present after deleting first two rows :  1658


# 1

In [6]:
%%file MovieOfGenre.py

from mrjob.job import MRJob 
import csv

class MovieOfGenre(MRJob):

    def mapper(self, _, line):
        row = next(csv.reader([line]))
        #This will split the values according to the column 
        Year, Length, Genre = row[0], row[1], row[3]
        if Year and Length and Genre:
            Year = int(Year)
            Length = int(Length)
            if Year > 1970 and Length > 75:
                yield Genre, 1
    
    def combiner(self, Genre, Value):
        yield Genre, sum(Value)
    
    def reducer(self, Genre, Value):
        yield Genre, sum(Value)
        
if __name__ == '__main__':
    MovieOfGenre.run()

Overwriting MovieOfGenre.py


In [24]:
!python MovieOfGenre.py Movies.csv

"Action"	154
"Adventure"	4
"Comedy"	274
"Drama"	409
"Fantasy"	1
"Horror"	44
"Music"	18
"Mystery"	66
"Romance"	1
"Science Fiction"	31
"War"	10
"Western"	23
"Westerns"	3


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ABDULL~1\AppData\Local\Temp\MovieOfGenre.Abdullah Awan.20230325.203111.916027
Running step 1 of 1...
job output is in C:\Users\ABDULL~1\AppData\Local\Temp\MovieOfGenre.Abdullah Awan.20230325.203111.916027\output
Streaming final output from C:\Users\ABDULL~1\AppData\Local\Temp\MovieOfGenre.Abdullah Awan.20230325.203111.916027\output...
Removing temp directory C:\Users\ABDULL~1\AppData\Local\Temp\MovieOfGenre.Abdullah Awan.20230325.203111.916027...


# 2

In [8]:
%%file ActressMovie.py

from mrjob.job import MRJob
import csv

class ActressMovie(MRJob):

    def mapper(self, _, line):
        row = next(csv.reader([line]))
        Year, Title, Actress = row[0], row[2], row[5]
        if Actress and Title and Year:
            yield Actress, (Title, Year)

    def combiner(self, Actress, Value):
        count = set(tuple(year) for year in Value)
        for year in count:
            yield Actress, year

    def reducer(self, Actress, Value):
        movies = set(tuple(year) for year in Value)
        yield Actress, list(movies)

if __name__ == '__main__':
    ActressMovie.run()

Overwriting ActressMovie.py


In [9]:
!python ActressMovie.py Movies.csv

"Abril, Victoria"	[["Tie Me Up! Tie Me Down!", "1990"], ["High Heels", "1991"]]
"Adams, Brooke"	[["Cuba", "1979"], ["Dead Zone, The", "1983"], ["Days of Heaven", "1978"]]
"Adams, Maud"	[["Octopussy", "1983"], ["Target Eagle", "1984"]]
"Adams, Trudy"	[["American Angels: Baptism of Blood, The", "1989"]]
"Adjani, Isabelle"	[["Subway", "1985"], ["Camille Claudel", "1990"]]
"Adolphson, Kristina"	[["Fanny and Alexander", "1982"]]
"Aimee, Anouk"	[["Un Hombre y una Mujer", "1966"], ["A Man & a Woman", "1966"], ["A Man & a Woman: Twenty Years Later", "1986"], ["Tragedy of a Ridiculous Man", "1982"]]
"Aleandro, Norma"	[["Official Story, The", "1985"]]
"Alexander, Denise"	[["Lindbergh Kidnapping Case, The", "1976"]]
"Algood, Sara"	[["Blackmail", "1929"]]
"Allen, Elizabeth"	[["Donovan's Reef", "1963"]]
"Allen, Joan"	[["Tucker: The Man & His Dream", "1988"]]
"Allen, Karen"	[["Scrooged", "1988"], ["Raiders of the Lost Ark", "1981"]]
"Alonso, Maria Conchita"	[["Predator 2", "1991"], ["Colors", "1988"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ABDULL~1\AppData\Local\Temp\ActressMovie.Abdullah Awan.20230325.202957.110906
Running step 1 of 1...
job output is in C:\Users\ABDULL~1\AppData\Local\Temp\ActressMovie.Abdullah Awan.20230325.202957.110906\output
Streaming final output from C:\Users\ABDULL~1\AppData\Local\Temp\ActressMovie.Abdullah Awan.20230325.202957.110906\output...
Removing temp directory C:\Users\ABDULL~1\AppData\Local\Temp\ActressMovie.Abdullah Awan.20230325.202957.110906...


# 3

In [10]:
%%file TopPopularMovies.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

class TopPopularMovies(MRJob):

    def mapper(self, _, line):
        row = next(csv.reader([line]))
        Year, Title, Popularity = row[0], row[2], row[7]
        if Title and Popularity and Year:
            Year = int(Year)
            Popularity = int(Popularity)
            if Year >= 1950 and Year <= 1990:
                yield Title, Popularity

    def combiner(self, Title, Value):
        yield Title, sum(Value)

    def reducer(self, Title, Value):
        yield None, (sum(Value), Title)
        #sorted_values = sorted(Value, reverse=True)
        #for i in range(min(len(sorted_values), 25)):
        #for i in range (0, length):
            #yield sorted_values[i]
        #yield Title, max(Value)
    
    def reducerTopMovies(self, _, Value):
        for count, Title in sorted(Value, reverse=True)[:25]:
            yield (Title, count)
            
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer),
            MRStep(reducer=self.reducerTopMovies)
        ]
        
if __name__ == '__main__':
    TopPopularMovies.run()

Overwriting TopPopularMovies.py


In [11]:
!python TopPopularMovies.py Movies.csv

"Time Machine, The"	88
"Terror, The"	88
"Out of Africa"	88
"New Year's Day"	88
"Let It Ride"	88
"Jeremiah Johnson"	88
"Head Office"	88
"Happy Birthday to Me"	88
"Guilty by Suspicion"	88
"Gonzo Presents Muppet Weird Stuff"	88
"French Connection, The"	88
"Five Corners"	88
"Final Notice"	88
"Fellini Satyricon"	88
"Dangerous Pursuit"	88
"Creature from Black Lake"	88
"Blood in, Blood Out"	88
"Best of Times, The"	88
"Ballad of Narayama, The"	88
"X, Y & Zee"	87
"Three Days of the Condor"	87
"Superman, The Movie"	87
"Star Trek V: The Final Frontier"	87
"Russkies"	87
"John Wayne Six Pack"	87


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ABDULL~1\AppData\Local\Temp\TopPopularMovies.Abdullah Awan.20230325.202958.550367
Running step 1 of 2...
Running step 2 of 2...
job output is in C:\Users\ABDULL~1\AppData\Local\Temp\TopPopularMovies.Abdullah Awan.20230325.202958.550367\output
Streaming final output from C:\Users\ABDULL~1\AppData\Local\Temp\TopPopularMovies.Abdullah Awan.20230325.202958.550367\output...
Removing temp directory C:\Users\ABDULL~1\AppData\Local\Temp\TopPopularMovies.Abdullah Awan.20230325.202958.550367...


# 4

In [12]:
%%file PopularMoviesOfActors.py

from mrjob.job import MRJob
from heapq import nlargest
import csv

class PopularMoviesOfActors(MRJob):

    def mapper(self, _, line):
        row = next(csv.reader([line]))
        Title, Actor, Popularity = row[2], row[4], row[7]
        if Actor and Title and Popularity:
            Popularity = int(Popularity)
            yield Actor, (Title, Popularity)

    def combiner(self, Actor, Value):
        movies = nlargest(5, Value)
        for movie in movies:
            yield Actor, movie

    def reducer(self, Actor, Value):
        movies = nlargest(5, Value)
        yield Actor, movies

if __name__ == '__main__':
    PopularMoviesOfActors.run()

Overwriting PopularMoviesOfActors.py


In [13]:
!python PopularMoviesOfActors.py Movies.csv

"Abel, Alfred"	[["Metropolis", 49]]
"Abraham, F. Murray"	[["Amadeus", 6]]
"Adolphson, Edvin"	[["Only One Night", 26], ["Count of Old Town, The", 72]]
"Aherne, Brian"	[["Lady in Question", 57]]
"Ahlstedt, B\u00f6rje"	[["Fanny and Alexander", 81]]
"Aiello, Danny"	[["Purple Rose of Cairo, The", 20], ["Do the Right Thing", 5]]
"Akan, Tarik"	[["Yol", 53]]
"Albaic\u00edn, Rafael"	[["Scarlet Letter, The", 55]]
"Albert, Edward"	[["Butterflies Are Free", 82]]
"Alda, Alan"	[["Sweet Liberty", 12], ["A New Life", 53]]
"Allen, Bill"	[["RAD", 75]]
"Allen, Woody"	[["Sleeper", 59], ["Scenes from a Mall", 8], ["Play It Again, Sam", 81], ["New York Stories", 6], ["Manhattan", 82]]
"Alterio, Hector"	[["Official Story, The", 39]]
"Ameche, Don"	[["Cocoon", 45]]
"Anderson, Kevin"	[["Miles from Home", 53]]
"Andersson, Wiktor"	[["Crisis", 66]]
"Anglade, Jean-Hughes"	[["Betty Blue", 71]]
"Annese, Frank"	[["House of the Rising Sun", 45]]
"Apfel, Oscar"	[["Inspiration", 66]]
"Arestrup, Niels"	[["Meeting Venus", 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ABDULL~1\AppData\Local\Temp\PopularMoviesOfActors.Abdullah Awan.20230325.203000.549708
Running step 1 of 1...
job output is in C:\Users\ABDULL~1\AppData\Local\Temp\PopularMoviesOfActors.Abdullah Awan.20230325.203000.549708\output
Streaming final output from C:\Users\ABDULL~1\AppData\Local\Temp\PopularMoviesOfActors.Abdullah Awan.20230325.203000.549708\output...
Removing temp directory C:\Users\ABDULL~1\AppData\Local\Temp\PopularMoviesOfActors.Abdullah Awan.20230325.203000.549708...


# 5

In [14]:
%%file AvgLenOfMovies.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

class AvgLenOfMovies(MRJob):
    
    def mapper(self, _, line):
        row = next(csv.reader([line]))
        Length, Genre = row[1], row[3]
        if Genre and Length:
            Length = int(Length)
            yield Genre, (Length, 1)

    def combiner(self, Genre, Value):
        #TotalLen = sum(values)
        #TotalNum = len(values)
        #yield genre, (total_length, num_movies)
        TotalLen = 0
        TotalNum = 0
        for length, count in Value:
            TotalLen += length
            TotalNum += count
        yield Genre, (TotalLen, TotalNum)

    def reducer(self, Genre, Value):
        TotalLen = 0
        TotalNum = 0
        for length, count in Value:
            TotalLen += length
            TotalNum += count
        yield Genre, TotalLen/TotalNum
        
if __name__ == '__main__':
    AvgLenOfMovies.run()

Overwriting AvgLenOfMovies.py


In [15]:
!python AvgLenOfMovies.py Movies.csv

"Action"	104.5
"Adventure"	119.0
"Comedy"	96.50540540540541
"Crime"	66.0
"Drama"	113.30455259026688
"Fantasy"	102.0
"Horror"	93.92727272727272
"Music"	100.48780487804878
"Mystery"	103.00990099009901
"Romance"	127.0
"Science Fiction"	106.47368421052632
"Short"	40.0
"War"	116.90625
"Western"	93.0091743119266
"Westerns"	124.8


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ABDULL~1\AppData\Local\Temp\AvgLenOfMovies.Abdullah Awan.20230325.203002.080302
Running step 1 of 1...
job output is in C:\Users\ABDULL~1\AppData\Local\Temp\AvgLenOfMovies.Abdullah Awan.20230325.203002.080302\output
Streaming final output from C:\Users\ABDULL~1\AppData\Local\Temp\AvgLenOfMovies.Abdullah Awan.20230325.203002.080302\output...
Removing temp directory C:\Users\ABDULL~1\AppData\Local\Temp\AvgLenOfMovies.Abdullah Awan.20230325.203002.080302...


# 6

In [16]:
%%file LeadActors.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

class LeadActors(MRJob):
    
    def mapper(self, _, line):
        row = next(csv.reader([line]))
        Actor, Actress = row[4], row[5]
        if Actor and Actress:
            yield (Actor, Actress), 1
    
    def combiner(self, key, Value):
        yield key, sum(Value)
    
    def reducer(self, key, Value):
        count = sum(Value)
        if count > 1:
            yield key, count

if __name__ == '__main__':
    LeadActors.run()

Overwriting LeadActors.py


In [17]:
!python LeadActors.py Movies.csv

["Adolphson, Edvin", "Bergman, Ingrid"]	2
["Allen, Woody", "Farrow, Mia"]	2
["Allen, Woody", "Keaton, Diane"]	5
["Banderas, Antonio", "Maura, Carmen"]	2
["Bj\u00f6rnstrand, Gunnar", "Andersson, Harriet"]	3
["Brando, Marlon", "Simmons, Jean"]	2
["Brown, Johnny Mack", "O'Day, Nell"]	3
["Burton, Richard", "Taylor, Elizabeth"]	3
["Chase, Chevy", "Hawn, Goldie"]	2
["Cleese, John", "Scales, Prunella"]	2
["De Niro, Robert", "Streep, Meryl"]	2
["Depardieu, G\u00e9rard", "Deneuve, Catherine"]	2
["Dillon, Matt", "Lane, Diane"]	2
["Douglas, Melvyn", "Garbo, Greta"]	2
["Douglas, Michael", "Turner, Kathleen"]	2
["Eastwood, Clint", "Bloom, Verna"]	2
["Eastwood, Clint", "Locke, Sondra"]	2
["Eastwood, Clint", "Mason, Marsha"]	2
["Ford, Glenn", "Hayworth, Rita"]	3
["Gere, Richard", "Basinger, Kim"]	2
["Gilbert, John", "Garbo, Greta"]	3
["Grant, Cary", "Bergman, Ingrid"]	2
["Hamill, Mark", "Fisher, Carrie"]	3
["Hogan, Paul", "Kozlowski, Linda"]	2
["Hopkins, Anthony", "Bancroft, Anne"]	2
["Mastroianni, M

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ABDULL~1\AppData\Local\Temp\LeadActors.Abdullah Awan.20230325.203003.638560
Running step 1 of 1...
job output is in C:\Users\ABDULL~1\AppData\Local\Temp\LeadActors.Abdullah Awan.20230325.203003.638560\output
Streaming final output from C:\Users\ABDULL~1\AppData\Local\Temp\LeadActors.Abdullah Awan.20230325.203003.638560\output...
Removing temp directory C:\Users\ABDULL~1\AppData\Local\Temp\LeadActors.Abdullah Awan.20230325.203003.638560...


# 7

In [18]:
%%file SpecificDirectors.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

class SpecificDirectors(MRJob):

    def mapper(self, _, line):
        row = next(csv.reader([line]))
        Genre, Director = row[3], row[6]
        if Director and Genre:
            yield Director, Genre

    def combiner(self, Director, Value):
        #yield pair, sum(counts)
        yield Director, ','.join(Value)

    def reducer(self, Director, Value):
        #Director, Genre = key
        #genre_counts = {'Drama': 0, 'Mystery': 0}
        #for count in Value:
            #genre_counts[Genre] += count
        #if genre_counts['Drama'] > 0 and genre_counts['Mystery'] > 0:
            #yield Director, genre_counts
        genres_list = ','.join(Value)
        if 'Mystery' in genres_list and 'Drama' in genres_list:
            yield Director, genres_list  

if __name__ == '__main__':
    SpecificDirectors.run()

Overwriting SpecificDirectors.py


In [19]:
!python SpecificDirectors.py Movies.csv

"Aldrich, Robert"	"Mystery,Action,Western,Drama,Comedy"
"Attenborough, Richard"	"Mystery,Drama,Music"
"Avildsen, John G."	"Mystery,Drama,Drama"
"Benton, Robert"	"Comedy,Drama,Drama,Mystery"
"Cimino, Michael"	"Mystery,Drama,Drama,Action"
"Demme, Jonathan"	"Mystery,Drama,Comedy"
"Eastwood, Clint"	"Western,Drama,Action,Adventure,Westerns,Action,Drama,War,Action,Comedy,Western,Mystery,Drama,Action"
"Foley, James"	"Drama,Mystery"
"Hitchcock, Alfred"	"Mystery,Mystery,Mystery,Drama,Drama,Mystery,Mystery,Drama,Mystery,Mystery,Drama,Crime,Drama,Mystery,Mystery,Mystery,Horror,Mystery,Comedy,Mystery,Drama,Drama,Drama,Mystery,Mystery,Mystery,Drama,Mystery,Drama,Science Fiction,Mystery,Mystery,Mystery,Mystery"
"Kagan, Jeremy Paul"	"Mystery,Drama"
"Kubrick, Stanley"	"Horror,Mystery,Drama,Drama,Drama,Science Fiction,Drama,Science Fiction,War"
"Kurosawa, Akira"	"Adventure,Drama,Drama,Drama,Drama,Drama,Drama,Drama,Mystery,Drama,Drama,Action,Drama,Drama"
"Lang, Fritz"	"Drama,Mystery,Drama,Drama,Science 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ABDULL~1\AppData\Local\Temp\SpecificDirectors.Abdullah Awan.20230325.203005.248014
Running step 1 of 1...
job output is in C:\Users\ABDULL~1\AppData\Local\Temp\SpecificDirectors.Abdullah Awan.20230325.203005.248014\output
Streaming final output from C:\Users\ABDULL~1\AppData\Local\Temp\SpecificDirectors.Abdullah Awan.20230325.203005.248014\output...
Removing temp directory C:\Users\ABDULL~1\AppData\Local\Temp\SpecificDirectors.Abdullah Awan.20230325.203005.248014...


# 8

In [20]:
%%file DirectorDetails.py

from mrjob.job import MRJob
from statistics import mean
import csv

class DirectorDetails(MRJob):
    
    def mapper(self, _, line):
        row = next(csv.reader([line]))
        Director, Popularity = row[6], row[7]
        if Director and Popularity:
            Popularity = int(Popularity)
            yield Director, Popularity
    
    def combiner(self, Director, Popularity):
        yield Director, list(Popularity)
    
    def reducer(self, Director, Popularity):
        Ranking = [r for sublist in Popularity for r in sublist]
        AvgRanking = mean(Ranking)
        MaxRanking = max(Ranking)
        MinRanking = min(Ranking)
        yield Director, (AvgRanking, MaxRanking, MinRanking)
        
if __name__ == '__main__':
    DirectorDetails.run()

Overwriting DirectorDetails.py


In [21]:
!python DirectorDetails.py Movies.csv

"Abrahams, Jim"	[62.5, 84, 41]
"Adolphson, Edvin"	[72, 72, 72]
"Alda, Alan"	[32.5, 53, 12]
"Aldrich, Robert"	[62.2, 84, 10]
"Allegret, Marc"	[25, 25, 25]
"Allen, Woody"	[37.625, 82, 7]
"Almod\u00f3var, Pedro"	[62.333333333333336, 73, 34]
"Altman, Robert"	[11, 19, 3]
"Amateau, Rod"	[80, 80, 80]
"Amurri, Franco"	[19, 19, 19]
"Anderson, Michael"	[34, 34, 34]
"Anger, Kenneth"	[62, 62, 62]
"Annakin, Ken"	[7, 7, 7]
"Annaud, Jean-Jacques"	[16, 24, 8]
"Anspaugh, David"	[2, 2, 2]
"Anthony, Joseph"	[44, 67, 21]
"Antonio, Lou"	[54, 54, 54]
"Antonioni, Michelangelo"	[32, 32, 32]
"Apted, Michael"	[32, 41, 23]
"Arkush, Allan"	[34, 34, 34]
"Arnold, Newt"	[78, 78, 78]
"Ashby, Hal"	[50, 69, 31]
"Aslanian, Samson"	[8, 8, 8]
"Attenborough, Richard"	[54.333333333333336, 85, 7]
"Auer, John H."	[36, 36, 36]
"August, Bille"	[14, 14, 14]
"Aurel, Jean"	[66, 66, 66]
"Avedis, Howard Hikmet"	[74, 74, 74]
"Avildsen, John G."	[70.33333333333333, 82, 51]
"Avnet, John"	[52, 52, 52]
"Axel, Gabriel"	[79, 79, 79]
"Aykro

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ABDULL~1\AppData\Local\Temp\DirectorDetails.Abdullah Awan.20230325.203006.634813
Running step 1 of 1...
job output is in C:\Users\ABDULL~1\AppData\Local\Temp\DirectorDetails.Abdullah Awan.20230325.203006.634813\output
Streaming final output from C:\Users\ABDULL~1\AppData\Local\Temp\DirectorDetails.Abdullah Awan.20230325.203006.634813\output...
Removing temp directory C:\Users\ABDULL~1\AppData\Local\Temp\DirectorDetails.Abdullah Awan.20230325.203006.634813...


# Q2

In [22]:
%%file FindingSimilarMovies.py

from mrjob.job import MRJob
import csv

class FindingSimilarMovies(MRJob):

    def mapper(self, _, line):
        row = next(csv.reader([line]))
        Year, Length, Title, Genre, Actor, Actress, Director, Popularity, Awards, Image = row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9] 
        if row:
            Year = int(Year)
            if 1960 <= Year <= 1969:
                yield None, (row, 1)
            elif Year >= 1970: 
                yield None, (row, 2)

    def reducer(self, _, values):
        movies = []
        for movie in values:
            movies.append(movie)
        for i in range(len(movies)):
            movie1, type1 = movies[i]
            for j in range(i+1, len(movies)):
                movie2, type2 = movies[j]
                if type1 != type2:
                    if (movie1[3] == movie2[3]) and (movie1[6] == movie2[6]) and ((movie1[4] == movie2[4]) or (movie1[5] == movie2[5])):
                        yield movie1[2], movie2[2]

if __name__ == '__main__':
    FindingSimilarMovies.run()

Overwriting FindingSimilarMovies.py


In [23]:
!python FindingSimilarMovies.py Movies.csv

"A Man & a Woman"	"A Man & a Woman: Twenty Years Later"
"A Man & a Woman: Twenty Years Later"	"Un Hombre y una Mujer"
"Through a Glass Darkly"	"Cries & Whispers"
"Goldfinger"	"Diamonds are Forever"
"Party, The"	"A Shot in the Dark"
"Gardens of Stone"	"Rain People, The"
"Partner"	"Conformist, The"
"Persona"	"Scenes from a Marriage"
"Scenes from a Marriage"	"Hour of the Wolf"
"Scenes from a Marriage"	"Passion of Anna, The"
"A Fistful of Dollars"	"For a Few Dollars More"
"Glass Menagerie, The"	"Rachel, Rachel"
"Harry & Son"	"Rachel, Rachel"
"Fugitive Kind, The"	"Mr. & Mrs. Bridge"
"Last Emperor, The"	"Grim Reaper, The"
"Rhapsody in August"	"Bad Sleep Well, The"
"Bad Sleep Well, The"	"Kagemusha"
"Midnight Cowboy"	"Falcon & the Snowman, The"
"Mutiny on the Bounty"	"Complete Dirty Harry, Magnum Force, The"
"Mutiny on the Bounty"	"Dead Pool, The"
"Mutiny on the Bounty"	"Great White Death"
"Mutiny on the Bounty"	"Crimson Pirate, The"
"Mutiny on the Bounty"	"Train, The"
"Mutiny on the Bounty"	"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ABDULL~1\AppData\Local\Temp\FindingSimilarMovies.Abdullah Awan.20230325.203008.118015
Running step 1 of 1...
job output is in C:\Users\ABDULL~1\AppData\Local\Temp\FindingSimilarMovies.Abdullah Awan.20230325.203008.118015\output
Streaming final output from C:\Users\ABDULL~1\AppData\Local\Temp\FindingSimilarMovies.Abdullah Awan.20230325.203008.118015\output...
Removing temp directory C:\Users\ABDULL~1\AppData\Local\Temp\FindingSimilarMovies.Abdullah Awan.20230325.203008.118015...
